In [1]:
import numpy as np
import matplotlib as plt
import PyQt5
import heapq
import time

from collections import deque

In [2]:
class Node:
    
    def __init__(self, state, depth, position = None, parent = None):
        self.cost = 0
        self.state = state
        self.depth = depth
        self.children = []
        self.position = position
        self.parent = parent
           
    def __lt__(self, other):
        op_pr = {'Up': 0, 'Down': 1, 'Left': 2, 'Right': 3}
        return op_pr[self.position] < op_pr[other.position]
            
    def print_grid(self):
        state = self.state
        grid = "| " + str(state[0]) + " | " + str(state[1]) + " | " + str(state[2]) + " |\n" +  "| " + str(state[3]) + " | " + str(state[4]) + " | " + str(state[5]) + " |\n" + "| " + str(state[6]) + " | " + str(state[7]) + " | " + str(state[8]) + " |\n"
        print(grid)
        
    def goal_reached(self):
        if self.state == [0, 1, 2, 3, 4, 5, 6, 7, 8]:
            return True
        else:
            return False
        
    def find_zero(self):
        for i in range(0, 9):
            if self.state[i] == 0:
                return i
    
    def up_move(self):
        i = self.find_zero()
        if i == 0 or i == 1 or i == 2:
            return False
        else:
            return True
    
    def down_move(self):
        i = self.find_zero()
        if i == 6 or i == 7 or i == 8:
            return False
        else:
            return True
        
    def right_move(self):
        i = self.find_zero()
        if i == 2 or i == 5 or i == 8:
            return False
        else:
            return True
        
    def left_move(self):
        i = self.find_zero()
        if i == 0 or i == 3 or i == 6:
            return False
        else:
            return True
        
    def find_children(self):
                
        if self.right_move():
            i = self.find_zero()
            child_right_state = self.state.copy()
            child_right_state[i] = child_right_state[i+1]
            child_right_state[i+1] = 0
            
            self.children.append(Node(state = child_right_state, depth = self.depth + 1, position = "Right", parent = self))
            
        if self.left_move():
            i = self.find_zero()
            child_left_state = self.state.copy()
            child_left_state[i] = child_left_state[i-1]
            child_left_state[i-1] = 0
            
            self.children.append(Node(state = child_left_state, depth = self.depth + 1, position = "Left", parent = self))
        
        if self.down_move():
            i = self.find_zero()
            child_down_state = self.state.copy()
            child_down_state[i] = child_down_state[i+3]
            child_down_state[i+3] = 0
            
            self.children.append(Node(state = child_down_state, depth = self.depth + 1, position = "Down", parent = self))
        
        if self.up_move():
            i = self.find_zero()
            child_up_state = self.state.copy()
            child_up_state[i] = child_up_state[i-3]
            child_up_state[i-3] = 0
            
            self.children.append(Node(state = child_up_state, depth = self.depth + 1, position = "Up", parent = self))
            
    def manhattan_distance(self):
        state = np.array(self.state)
        goal = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
        
        x_cost = abs(state % 3 - goal % 3)
        y_cost = abs(state // 3 - goal // 3)
        
        total_cost = x_cost + y_cost
        
        return sum(total_cost[1:])
    
    def euclidean_distance(self):
        state = np.array(self.state)
        goal = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
        
        x_cost = abs(state % 3 - goal % 3)
        y_cost = abs(state // 3 - goal //3)
        
        total_cost = np.sqrt(x_cost**2 + y_cost**2)
        
        return sum(total_cost[1:])
    

In [5]:
class Solver:
    
    def __init__(self):
        self.node = None
        self.expanded = []
        self.time = None
    
    def print_results(self, node, cost_type):
        path = [node]
        node.parent.print_grid()
        cost = 0
        
        while not node.parent == None:
            path.append(node.parent)
            node = node.parent
        path = path[::-1]
        
        print("________________________________________________________\n")
        
        for i in range(0, len(path)):
            print("Node Depth: ", i, "\nNode Direction: ", path[i].position)
            path[i].print_grid()
            if cost_type == "Manhatten":
                cost += 1 + path[i].manhattan_distance()
            elif cost_type == "Euclidean":
                cost += 1 + path[i].euclidean_distance()
            else:
                cost += 1
        
        print("\nPath Length:\t", len(path))
        print("Total Cost:\t", cost, cost_type)
        print("Search Depth:\t", path[len(path) - 1].depth)
        print("Nodes Expanded:\t", len(self.expanded))
        print("Running Time:\t", self.time)
        
    def contains(self, child, frontier, explored, with_cost = True):
        f_size = len(frontier)
        e_size = len(explored)
        if with_cost:
            for i in range(0,max(f_size, e_size)):
                if i < f_size:
                    if child.state == frontier[i][1].state:
                        return True, frontier[i][1]
                if i < e_size:
                    if child.state == explored[i].state:
                        return True, explored[i]
        else:
            for i in range(0,max(f_size, e_size)):
                if i < f_size:
                    if child.state == frontier[i].state:
                        return True, frontier[i][1]
                if i < e_size:
                    if child.state == explored[i].state:
                        return True, explored[i]
        return False, None
    
    def DFS_search(self, initial_state):
                
        frontier = []
        explored = []

        frontier = deque()
        frontier.append(initial_state)
        
        step = 0
        start_time = time.time()
        
        while frontier:
            
            node = frontier.pop()
            explored.append(node)

            node.find_children()
            
            step = step + 1
            print("\nStep: ", str(step), node.position)
            
            node.print_grid()
            
            if node.goal_reached():
                self.node = node
                self.expanded = explored
                self.time = time.time() - start_time
                return True, node
            
            for child in node.children:
                if not self.contains(child, frontier, explored, False):
                    frontier.append(child)
                    
        return False, None
    
    def BFS_Search(self, initial_state, cost_type):
                
        frontier = []
        explored = []

        frontier = deque()
        frontier.append(initial_state)
        
        step = 0
        start_time = time.time()
        
        while frontier:
            
            node = frontier.pop(0)
            explored.append(node)

            node.find_children()
            
            step = step + 1
            print("\nStep: ", str(step), node.position)
            
            node.print_grid()
            
            if node.goal_reached():
                self.node = node
                self.expanded = explored
                self.time = time.time() - start_time
                return True, node
            
            for child in node.children:
                if not self.contains(child, frontier, explored, False):
                    frontier.append(child)
                    
        return False, None
    
    def A_search(self, initial_state, cost_type):
        frontier = []
        explored = []
        
        initial_state.cost = 0
        
        if cost_type == "Manhatten":
            initial_state.cost = initial_state.manhattan_distance()
        elif cost_type == "Euclidean":
            initial_state.cost = initial_state.euclidean_distance()
        
        heapq.heappush(frontier, (initial_state.cost, initial_state))
        
        step = 0
        start_time = time.time()
        
        while frontier:
            
            node = heapq.heappop(frontier)[1]
            explored.append(node)
            
            node.find_children()
            
            step = step + 1
            print("\nStep: ", str(step), node.position)
            print("Depth: ", str(node.depth))
            
            node.print_grid()
            
            if node.goal_reached():
                self.node = node
                self.expanded = explored
                self.time = time.time() - start_time
                return True, node
            
            for child in node.children:
                if cost_type == "Manhatten":
                    child.cost = child.manhattan_distance()
                elif cost_type == "Euclidean":
                    child.cost = child.euclidean_distance()
                
                condition, repeated_node = self.contains(child, frontier, explored)

                if not condition:
                    heapq.heappush(frontier, (child.cost, child))
                else:
                    child.depth = repeated_node.depth

        return False, None

In [6]:
initial_state = Node(state = [1, 2, 5, 3, 4, 0, 6, 7, 8], depth = 0)

solver = Solver()

# is_reached, goal = solver.A_search(initial_state, "Manhatten")

# if (is_reached):
#     solver.print_results(goal, "Manhatten")

is_reached, goal = solver.A_search(initial_state, "Euclidean")

if (is_reached):
    solver.print_results(goal, "Euclidean")
    
# is_reached, goal = solver.DFS_search(initial_state)

# if (is_reached):
#     solver.print_results(goal)



Step:  1 None
Depth:  0
| 1 | 2 | 5 |
| 3 | 4 | 0 |
| 6 | 7 | 8 |


Step:  2 Up
Depth:  1
| 1 | 2 | 0 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


Step:  3 Left
Depth:  2
| 1 | 0 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


Step:  4 Left
Depth:  3
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

| 1 | 0 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

________________________________________________________

Node Depth:  0 
Node Direction:  None
| 1 | 2 | 5 |
| 3 | 4 | 0 |
| 6 | 7 | 8 |

Node Depth:  1 
Node Direction:  Up
| 1 | 2 | 0 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

Node Depth:  2 
Node Direction:  Left
| 1 | 0 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

Node Depth:  3 
Node Direction:  Left
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


Path Length:	 4
Total Cost:	 12.23606797749979 Euclidean
Search Depth:	 3
Nodes Expanded:	 4
Running Time:	 0.0019948482513427734
